# Demo -- Model Evaluation



In [1]:
import logging
import os

from cjw.knowledgeqa import indexers, bots
from cjw.knowledgeqa.evaluators.ConsistencyEvaluator import ConsistencyEvaluator
from cjw.knowledgeqa.evaluators.ProximityEvaluator import ProximityEvaluator
from cjw.knowledgeqa.evaluators.QAData import QAData
from cjw.utilities.embedding.BertEmbedding import BertEmbedding


[nltk_data] Downloading package punkt to /Users/cjwang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Set up the Environment

Adjust these variables if necessary to suit the local environment.


In [2]:

HOME = os.path.expanduser("~")
PROJECT_DIR = f"{HOME}/IdeaProjects/knowledgeqa"
DATA_FILE = f"{PROJECT_DIR}/data/wikipedia_question_similar_answer.tsv"

MARQO_SERVER = 'http://localhost:8882'
TEST_INDEX_NAME = "wiki_test_qa"

RAG_KNOWLEDGE = 10       # How many facts should be pulled out from the index before being fed to LLM
GPT_TEMPERATURE = 0.6

In [3]:
# Turn on logger too see the internal work

# logging.basicConfig(level=logging.INFO)


## Load Data, Indexing, and Create the Bot under Evaluation

In [4]:
# Read the golden Q&A data
data = QAData(DATA_FILE)

In [5]:
# Start an index to hold the Q&A data
index = indexers.index("marqo", new=True, serverUrl=MARQO_SERVER, indexName=TEST_INDEX_NAME)

In [6]:
# Index the data if it is not yet in there
if index.size() == 0:
    await index.add(data.to_dict(), keyFields=["answer"])

/var/folders/mj/8vz4yhqj5w76hnkw54qktw900000gn/T/ipykernel_85438/787393499.py:2: RuntimeWarning: coroutine 'MarqoIndexer.size' was never awaited
  if index.size() == 0:


In [7]:
# Test indexer's answer
question = "what is singapore's currency"
indexerAnswers = await index.search(question, top=3)
indexerAnswers

[{'question': "what is singapore's currency",
  'answer': 'the singapore dollar or dollar ( sign : $; code : sgd) is the official currency of singapore .',
  '_id': '1377',
  '_highlights': {'answer': 'the singapore dollar or dollar ( sign : $; code : sgd) is the official currency of singapore .'},
  '_score': 0.90729886},
 {'question': 'what is korean money called',
  'answer': 'the won () ( sign : ₩; code : krw) is the currency of south korea .',
  '_id': '1429',
  '_highlights': {'answer': 'the won () ( sign : ₩; code : krw) is the currency of south korea .'},
  '_score': 0.7079103},
 {'question': 'who composed the singapore national anthem',
  'answer': "composed by zubir said in 1958 as a theme song for official functions of the city council of singapore, the song was selected in 1959 as the island's anthem when it attained self-government .",
  '_id': '678',
  '_highlights': {'answer': "composed by zubir said in 1958 as a theme song for official functions of the city council of s

In [8]:
# Create a bot using the Q&A contents in the index
bot = bots.bot("gpt4").withFacts(index, contentFields=["answer"], top=RAG_KNOWLEDGE)

In [9]:
# Test the bot.  (Note the multiple citations.)
question = "what does the president of the usa do"
botAnswer = await bot.ask(question)
print(botAnswer)

print("In the index")
articles = await index.get(botAnswer.citations)
for a in articles:
    print(f"[{a['_id']}] {a['answer']}")

The President of the United States of America is the head of state and head of government of the United States, leading the executive branch of the federal government and acting as the commander-in-chief of the United States armed forces. The President also has the power to grant federal pardons and reprieves, and to convene and adjourn either or both houses of Congress under extraordinary circumstances. [409,410,411]
In the index
[409] the president of the united states of america (potus) is the head of state and head of government of the united states .
[410] the president leads the executive branch of the federal government and is the commander-in-chief of the united states armed forces .
[411] the president is further empowered to grant federal pardons and reprieves , and to convene and adjourn either or both houses of congress under extraordinary circumstances.


## By Finding Standard Answers in Proximity

Make the bot answer a question, embed the answer, and see if the standard answer is in the top N items in the proximity.

In [10]:
# Create the Evaluator for the bot
evaluator = await ProximityEvaluator().forBot(bot).withData(data, index)

score = await evaluator.evaluate(sampleSize=10, showFailedQuestions=True)

print(score)

1.0


## By Comparing Similarities with Repeating Questions

If we don't have a standard set, we can ask the bot multiple time with the same question.  If the bot is hallucinating, it is unlikely to hallucinate the same way every time.

In [11]:
questions = data.getQuestions()
bert = BertEmbedding("distilbert-multilingual-nli-stsb-quora-ranking")
evaluator = ConsistencyEvaluator(questions, bert).forBot(bot)

score = await evaluator.evaluate(sampleSize=3)

print(score)

0.9997768600781759


## Interesting Questions

I found the following questions/answers interesting in the provided Wikipedia data.

During my tests, I found the bot stated that it does not know "what states does interstate 70 travel through".  I checked the test data and found the answer to the question was:

*"interstate 70 (i-70) is an interstate highway in the united states that runs from interstate 15 near cove fort, utah , to a park and ride near baltimore, maryland ."*

This is **NOT** a correct answer for the question.  Thus, the bot is actually correct not knowing the answer from the provided facts.

In [12]:
answerUnknownQuestion = "what states does interstate 70 travel through"
answerUnknownId = '777'
print(await index.get(answerUnknownId))

print(await bot.ask(answerUnknownQuestion))

[{'_found': True, 'question': 'what states does interstate 70 travel through', 'answer': 'interstate 70 (i-70) is an interstate highway in the united states that runs from interstate 15 near cove fort, utah , to a park and ride near baltimore, maryland .', '_id': '777'}]
I don't know [--]


In [13]:
# These questions could not be found in the top-5 picks in the index.  When I choose top 10, they went a way.

answerNotFoundByIndex = "what age group is generation x"
# answerNotFoundByIndex = "what cards do you need in poker to get a royal flush"
# answerNotFoundByIndex = "what is the title of hobbes main work"
print(await bot.ask(answerNotFoundByIndex))

Generation X is defined by those born from the early 1960s to the early 1980s. [320]
